In [ ]:
# @title
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.saving import register_keras_serializable
from keras.models import load_model

In [ ]:
# @title
# Load your custom dataset
data = pd.read_csv('Dataset.csv')

# Map sentiment labels to integers
label_map = {
    "Satisfied": 0,
    "Neutral": 1,
    "Slightly Dissatisfied": 2,
    "Dissatisfied": 3,
    "Highly Dissatisfied": 4
}
data['label'] = data['label'].map(label_map)



In [ ]:
# @title
print(data['text'].isna().sum())
print(data['label'].isna().sum())

0
10


In [ ]:
# @title
data = data.dropna(subset=['text', 'label'])

In [ ]:
# @title
print(data['text'].isna().sum())
print(data['label'].isna().sum())

0
0


In [ ]:
# @title
# Perform stratified train-validation-test split (80-10-10 split)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42, stratify=data['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)



In [ ]:
# @title
# Verify class distribution
print("Class distribution in training set:")
print(pd.Series(train_labels).value_counts(normalize=True))

print("Class distribution in validation set:")
print(pd.Series(val_labels).value_counts(normalize=True))

print("Class distribution in test set:")
print(pd.Series(test_labels).value_counts(normalize=True))

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")



Class distribution in training set:
label
1.0    0.213511
3.0    0.208580
4.0    0.207840
2.0    0.186391
0.0    0.183679
Name: proportion, dtype: float64
Class distribution in validation set:
label
1.0    0.213018
3.0    0.209073
4.0    0.207101
2.0    0.187377
0.0    0.183432
Name: proportion, dtype: float64
Class distribution in test set:
label
1.0    0.214567
4.0    0.208661
3.0    0.208661
2.0    0.185039
0.0    0.183071
Name: proportion, dtype: float64


In [ ]:
# @title
def tokenize(texts, labels):
    tokenized = tokenizer(list(texts), padding=True, truncation=True, return_tensors='tf')
    tokenized_inputs = {
        'input_ids': tf.cast(tokenized['input_ids'], dtype=tf.int32),
        'attention_mask': tf.cast(tokenized['attention_mask'], dtype=tf.int32),
        'token_type_ids': tf.cast(tokenized['token_type_ids'], dtype=tf.int32),
    }
    tokenized_labels = tf.convert_to_tensor(labels.astype(int), dtype=tf.int32)
    return tokenized_inputs, tokenized_labels


# Now try to tokenize again
train_data = tokenize(train_texts, train_labels)
val_data = tokenize(val_texts, val_labels)
test_data = tokenize(test_texts, test_labels)


train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data).batch(32)

In [ ]:
# @title
inputs, labels = train_data  # Unpack the tuple
print("input_ids dtype:", inputs['input_ids'].dtype)
print("attention_mask dtype:", inputs['attention_mask'].dtype)
print("token_type_ids dtype:", inputs['token_type_ids'].dtype)
print("label dtype:", labels.dtype)


input_ids dtype: <dtype: 'int32'>
attention_mask dtype: <dtype: 'int32'>
token_type_ids dtype: <dtype: 'int32'>
label dtype: <dtype: 'int32'>


In [ ]:
# @title
# Verify a batch from the train dataset
inp, out = next(iter(train_dataset))
print("Input batch:", inp)
print("Output labels:", out)

# Load the pre-trained BERT model
base_model = TFAutoModel.from_pretrained("bert-base-cased")

# Define a classification model using BERT
# Define a classification model using BERT
@register_keras_serializable(package="Custom")
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]  # Extract pooled output (last hidden state)
        return self.fc(x)

    def get_config(self):
        config = super(BERTForClassification, self).get_config()
        config.update({
            'num_classes': self.fc.units,
            'bert_model_name': self.bert.name_or_path
        })
        return config

    @classmethod
    def from_config(cls, config):
        bert_model = TFAutoModel.from_pretrained(config["bert_model_name"])
        return cls(bert_model=bert_model, num_classes=config["num_classes"])

# This will be used when creating and saving the model:
def create_model():
    base_model = TFAutoModel.from_pretrained("bert-base-cased")
    model = BERTForClassification(base_model, num_classes=5)

    # Specify input signatures
    input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
    token_type_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="token_type_ids")

    # Call the model with input signatures
    outputs = model({
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids
    })

    return model

classifier = create_model()


Input batch: {'input_ids': <tf.Tensor: shape=(32, 30), dtype=int32, numpy=
array([[  101,   146,  1444,  6043,  1389,  1665,  1114,  5321,  1126,
         8926,  1106,  1546,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1109,  3317,  1110,  1171,  1106,  1684,  6150,   119,
         5438,  1111,  1103,  3613,  8239,   106,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,   146,   112,   182,  1378,  1146,  1106,  1267,  1191,
         1139,  1231, 14703,  3276,  4566,  1110,  1971,  1223,  3189,
          119,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [  101,  1494,  1106, 24295,  1126,  3342,  1104,  1546,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# @title
# Compile the model
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Train the model
history = classifier.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=2, restore_best_weights=True
        )
    ]
)



Epoch 1/3
127/127 [==============================] - 65s 243ms/step - loss: 0.7383 - accuracy: 0.7392 - val_loss: 0.2978 - val_accuracy: 0.9093
Epoch 2/3
127/127 [==============================] - 28s 220ms/step - loss: 0.2268 - accuracy: 0.9248 - val_loss: 0.2471 - val_accuracy: 0.9053
Epoch 3/3
127/127 [==============================] - 28s 216ms/step - loss: 0.1246 - accuracy: 0.9625 - val_loss: 0.2027 - val_accuracy: 0.9250


In [ ]:
# @title
# Evaluate the model on the test dataset
test_loss, test_accuracy = classifier.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Manual input for classification
def classify_manual_input():
    while True:
        user_input = input("Enter a sentence to classify (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        inputs = tokenizer(user_input, return_tensors="tf", padding=True, truncation=True)
        prediction = classifier(inputs)[0].numpy()
        predicted_class = prediction.argmax()
        for key, value in label_map.items():
            if value == predicted_class:
                print(f"Predicted class: {key} (Confidence: {prediction[predicted_class]:.2f})")

classify_manual_input()

16/16 [==============================] - 4s 77ms/step - loss: 0.2197 - accuracy: 0.9193
Test Loss: 0.2196902334690094, Test Accuracy: 0.9192913174629211
Enter a sentence to classify (or type 'exit' to quit): amazingly bad
Predicted class: Dissatisfied (Confidence: 0.73)
Enter a sentence to classify (or type 'exit' to quit): pathetic
Predicted class: Highly Dissatisfied (Confidence: 0.89)
Enter a sentence to classify (or type 'exit' to quit): absolute shit
Predicted class: Highly Dissatisfied (Confidence: 0.96)
Enter a sentence to classify (or type 'exit' to quit): this was absolutely shit service
Predicted class: Highly Dissatisfied (Confidence: 0.90)
Enter a sentence to classify (or type 'exit' to quit): i'm amazed at how disastrous this is
Predicted class: Dissatisfied (Confidence: 0.49)
Enter a sentence to classify (or type 'exit' to quit): disastrous
Predicted class: Highly Dissatisfied (Confidence: 0.72)
Enter a sentence to classify (or type 'exit' to quit): hmm
Predicted class: N

In [ ]:
# @title
classifier.save("Nishanth_saved_model", save_format="tf")

In [ ]:
# @title
loaded_model = tf.keras.models.load_model(
    "Nishanth_saved_model",
    custom_objects={"BERTForClassification": BERTForClassification}
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# @title
test_loss, test_accuracy = loaded_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

16/16 [==============================] - 8s 93ms/step - loss: 0.2234 - accuracy: 0.9252
Test Loss: 0.2233937382698059, Test Accuracy: 0.9251968264579773


In [ ]:
# @title
import shutil
shutil.make_archive("/content/Nishanth_saved_model", 'zip', "/content/Nishanth_saved_model")


'/content/Nishanth_saved_model.zip'